In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')
import os
os.listdir('gdrive/My Drive')
folder = 'gdrive/My Drive/mechfopf'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
import pandas as pd
import numpy as np

In [0]:
os.chdir(folder)

In [0]:
data = pd.read_csv("article_data.csv")

In [0]:
pos_reviews = data['review'][data['label']==1]
neg_reviews = data['review'][data['label']==-1]
unsup_reviews = data['review'][data['label']==0]

In [0]:
import gensim
from sklearn.model_selection import train_test_split
import numpy as np
#use 1 for positive sentiment, 0 for negative
y = np.concatenate((np.ones(len(pos_reviews)), np.zeros(len(neg_reviews))))
x_train, x_test, y_train, y_test = train_test_split(np.concatenate((pos_reviews, neg_reviews)), y, test_size=0.2)

#Do some very minor text preprocessing
def cleanText(corpus):
    punctuation = """.,?!:;(){}[]"""
    corpus = [z.lower().replace('\n','') for z in corpus]
    corpus = [z.replace('<br />', ' ') for z in corpus]

    #treat punctuation as individual words
    for c in punctuation:
        corpus = [z.replace(c, ' %s '%c) for z in corpus]
    corpus = [z.split() for z in corpus]
    return corpus

x_train = cleanText(x_train)
x_test = cleanText(x_test)
unsup_reviews = cleanText(unsup_reviews)

In [0]:
import random
size = 1000
#instantiate our DM and DBOW models
model_dm = gensim.models.Doc2Vec(min_count=1, window=10, vector_size=size, sample=1e-3, negative=5, workers=3)

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [0]:
tot=[]
tot.append(x_train)
tot.append(x_test)
tot.append(unsup_reviews)
tot = np.concatenate(np.array(tot))

In [0]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(tot)]

In [0]:
#build vocab over all reviews
model_dm.build_vocab(documents)

In [0]:
#We pass through the data set multiple times, shuffling the training reviews each time to improve accuracy.
tot_train=[]
tot_train.append(x_train)
tot_train.append(unsup_reviews)
tot_train = np.concatenate(np.array(tot_train))

In [0]:
perm = np.random.permutation(tot_train.shape[0])
docs_train = [TaggedDocument(doc, [i]) for i, doc in enumerate(tot_train[perm])]

In [0]:
model_dm.train(docs_train, total_examples=model_dm.corpus_count, epochs=10)

In [0]:
docs_x_train = [TaggedDocument(doc, [i]) for i, doc in enumerate(x_train)]

In [0]:
#Get training set vectors from our models
def getVecs(model, corpus, size):
    vecs = [np.array(model[z.tags[0]]).reshape((1, size)) for z in corpus]
    return np.concatenate(vecs)

train_vecs = getVecs(model_dm, docs_x_train, size)

In [0]:
x_test=np.array(x_test)
perm = np.random.permutation(x_test.shape[0])
docs_test = [TaggedDocument(doc, [i]) for i, doc in enumerate(x_test[perm])]

In [0]:
model_dm.train(docs_test, total_examples=model_dm.corpus_count, epochs=model_dm.epochs)

In [0]:
# model_dm.save("model_dm_article.model")

In [0]:
#Construct vectors for test reviews
def getVecs(model, corpus, size):
    vecs=[]
    for z in corpus: 
        try:
            v = np.array(model[z.tags[0]]).reshape((1, size))
            vecs.append(v)
        except:
            pass
    return np.concatenate(vecs)
    
test_vecs = getVecs(model_dm, docs_test, model_dm.vector_size)

In [0]:
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier(loss='log', penalty='l1')
lr.fit(train_vecs, y_train)

In [0]:
lr.score(train_vecs, y_train)

In [0]:
print(f'Test Accuracy: {lr.score(test_vecs, y_test)}')